Add latex macros$$\newcommand{\V}[1]{{\boldsymbol{#1}}}\newcommand{mean}[1]{{\mathbb{E}\left[#1\right]}}\newcommand{var}[1]{{\mathbb{V}\left[#1\right]}}\newcommand{covar}[2]{\mathbb{C}\text{ov}\left[#1,#2\right]}\newcommand{corr}[2]{\mathbb{C}\text{or}\left[#1,#2\right]}\newcommand{argmin}{\mathrm{argmin}}\def\rv{z}\def\reals{\mathbb{R}}\def\pdf{\rho}\def\rvdom{\Gamma}\def\coloneqq{\colon=}\newcommand{norm}{\lVert #1 \rVert}\def\argmax{\operatorname{argmax}}\def\ai{\alpha}\def\bi{\beta}\newcommand{\dx}[1]{\;\mathrm{d}#1}$$

In [ ]:
%matplotlib inline


# Sparse Grid Interpolation

We will use the Cantilever Beam benchmark to illustrate how to use a sparse grid
as a surrogate for uncertainty quantification and sensitivity analysis

.. figure:: ../../source/figures/cantilever-beam.png
   :align: center

   Conceptual model of the cantilever-beam

.. table:: Uncertain variables
   :align: center

   =============== ========= =======================
   Uncertainty     Symbol    Prior
   =============== ========= =======================
   Yield stress    $R$ $N(40000,2000)$
   Young's modulus $E$ $N(2.9e7,1.45e6)$
   Horizontal load $X$ $N(500,100)$
   Vertical Load   $Y$ $N(1000,100)$
   =============== ========= =======================


.. table:: Design variables
   :align: center

   =============== ========= =======================
   Uncertainty     Symbol    Range
   =============== ========= =======================
   Width           $w$ $[1, 4]$
   Thickness       $t$ $[1, 4]$
   =============== ========= =======================

First we must load the benchmark


In [ ]:
import numpy as np
from pyapprox.benchmarks import setup_benchmark
from pyapprox.variables import combine_uncertain_and_bounded_design_variables
from pyapprox import surrogates
from pyapprox.analysis import visualize
from pyapprox import util
import matplotlib.pyplot as plt
benchmark = setup_benchmark('cantilever_beam')
np.random.seed(1)

The cantilever beam model is an optimization benchmark. Consequently it
defines random and design variables. All surrogates require random variable
information so we define a new variable for this purpose



In [ ]:
variable = combine_uncertain_and_bounded_design_variables(
    benchmark.variable, benchmark.design_variable)

So lets just approximate the contraint function.

We can set a maximum number of samples using the options dictionary



In [ ]:
options = {"max_nsamples": 1000}
approx = surrogates.adaptive_approximate(
    benchmark.constraint_fun, variable, "sparse_grid", options).approx

Plot the sparse grid samples with



In [ ]:
sparse_grid_samples = approx.get_samples()
util.plot_2d_samples(sparse_grid_samples, marker='o')
plt.show()

We can estimate the error in the surrogate using some validation samples.



In [ ]:
validation_samples = variable.rvs(100)
validation_values = benchmark.constraint_fun(validation_samples)
approx_values = approx(validation_samples)
error = np.linalg.norm(validation_values-approx_values, axis=0)/np.sqrt(
    validation_values.shape[0])
print(f"The RMSE error is {error}")

and building a kernel density estimator. Lets first just plot the marginal
PDFs of the output



In [ ]:
surrogate_samples = variable.rvs(10000)
approx_values = approx(surrogate_samples)
visualize.plot_qoi_marginals(approx_values)
plt.show()